# A3: A\*, IDS, and Effective Branching Factor

Bradley Pospeck

## Overview

In this assignment implementations of a recursive best-first search A\* algorithm and an iterative deepening search are used. A\* search is considered an informed or a heuristic search. The algorithm is told of an estimate of the total cost of the path to the goal and makes decisions on where to go based on those values.

    At some intermediate node, the 
      estimated cost of the solution path =
          the sum of the step costs so far from the start node to this node
             +
          an estimate of the sum of the remaining step costs to a goal

Let's label these as

   * $f(n) =$ estimated cost of the solution path through node $n$
   * $g(n) =$ the sum of the step costs so far from the start node to this node
   * $h(n) =$ an estimate of the sum of the remaining step costs to a goal

*heuristic function*: $h(n) =$ estimated cost of the cheapest path from state at node $n$ to a goal state.

For this assignment, 3 heuristics will be used. They are described below in "Heuristic Functions".

While running the algorithms an effective branching factor (EBF) will be found. This will be used to compare the 4 algorithms. The branching factor of a tree is the number of children at each node. If this is not the same for all nodes, then we can find an average, or "effective" branching factor. There are 2 formulas that can be used for this, the latter of which is faster to solve than the other:

(1) $$  1 + b + b^2 + \cdots + b^d$$

(2) $$ \frac{1-b^{d+1}}{1-b}$$

In both formulas $b$ stands for the efb, $d$ is the depth to the goal, and the result should be equal to the number of nodes explored.

**Note: I used bits and pieces from lecture notes 7 and 10 for this Overview section**

## Imports

In [1]:
import copy

## Classes

The class below was provided in Lecture 7: Informed Search. It's used by the A\* search algorithm

In [2]:
class Node:
    def __init__(self, state, f=0, g=0 ,h=0):
        self.state = state
        self.f = f
        self.g = g
        self.h = h
    def __repr__(self):
        return "Node(" + repr(self.state) + ", f=" + repr(self.f) + \
               ", g=" + repr(self.g) + ", h=" + repr(self.h) + ")"

## Search Functions

The A\* search algorithms were pulled directly from Informed search (Lecture 7). 

`aStarSearch` doesn't have its own heuristic functions. Those will be given to it so it doesn't need to worry about their implementations.

In [3]:
def aStarSearch(startState, actionsF, takeActionF, goalTestF, hF):
    """Recursive best-first search algorithm implementation for A*"""
    h = hF(startState)
    startNode = Node(state=startState, f=0+h, g=0, h=h)
    return aStarSearchHelper(startNode, actionsF, takeActionF, goalTestF, hF, float('inf'))

In [4]:
def aStarSearchHelper(parentNode, actionsF, takeActionF, goalTestF, hF, fmax):
    """Help to A* that does all of the heavy lifting"""
    global nNodes
    if goalTestF(parentNode.state):
        return ([parentNode.state], parentNode.g)
    ## Construct list of children nodes with f, g, and h values
    actions = actionsF(parentNode.state)
    if not actions:
        return ("failure", float('inf'))
    children = []
    for action in actions:
        nNodes +=1
        (childState,stepCost) = takeActionF(parentNode.state, action)
        h = hF(childState)
        g = parentNode.g + stepCost
        f = max(h+g, parentNode.f)
        childNode = Node(state=childState, f=f, g=g, h=h)
        children.append(childNode)
    while True:
        # find best child
        children.sort(key = lambda n: n.f) # sort by f value
        bestChild = children[0]
        if bestChild.f > fmax:
            return ("failure",bestChild.f)
        # next lowest f value
        alternativef = children[1].f if len(children) > 1 else float('inf')
        # expand best child, reassign its f value to be returned value
        result,bestChild.f = aStarSearchHelper(bestChild, actionsF, takeActionF, goalTestF,
                                        hF, min(fmax,alternativef))
        if result is not "failure":
            result.insert(0,parentNode.state)  
            return (result, bestChild.f) 

Effective branching factor is determined by the next 2 functions. `ebf` handles initial checks. These initial checks include base cases as well as iteratively finding a reasonable value for an upper bound on a binary search for the effective branching factor. Next `ebfBinSearch` takes those bounds from `ebf` and performs a binary search to try and find a $b$ such that the number of nodes is equal to:

$$ \frac{1-b^{d+1}}{1-b}$$

In [5]:
def ebfBinSearch(nNodes, depth, precision, lower, upper):
    """Performs binary search recursively to find and return the effective branching factor to ebf()"""
    mid = (upper + lower)/2
    if mid==1.0:
        n = 1+mid
    else:
        n = (1 - mid**(depth+1) )/ (1 - mid)
    if abs(n-nNodes) <= precision: 
        return mid
    elif n > nNodes:
        return ebfBinSearch(nNodes, depth, precision, lower, mid)
    elif n < nNodes:
        return ebfBinSearch(nNodes, depth, precision, mid, upper)

In [6]:
def ebf(nNodes, depth, precision=0.01):
    """Returns the effective branching factor found via binary search"""
    if nNodes==0 and depth==0:
        return 0.0
    elif nNodes ==1 and depth==0:
        return 1.0
    else:
        b = float(depth)
        if b==1.00: b=2.0
        while (1 - b**(depth+1))/(1-b) < nNodes:
            b *= 2
        bUpper = b
        bLower = 0.0 
        return ebfBinSearch(nNodes, depth, precision, bLower, bUpper)

The next 2 functions are used from my A2 solutions. They're simply a recursive definition of iterative deepening search (IDS). Function definitions have further descriptions of what they do.

In [7]:
def depthLimitedSearch(state, goalState, actionsF, takeActionF, depthLimit):
    """Recursive definition of a depth limited search. Attempts to find the 'goalState' from the 'state'. Uses passed in 
    functions 'actionsF' and 'takeActionF' to determine which actions are viable before iterating through those actions."""
    global nNodes
    if state == goalState:
        return []
    if depthLimit == 0:
        return 'cutoff'
    cutoffOccurred = False
    for action in actionsF(state):
        nNodes +=1
        childState,_ = takeActionF(state, action) #underscore is anonymous variable that won't be used: absorbs cost
        result = depthLimitedSearch(childState, goalState, actionsF, takeActionF, depthLimit-1)
        if result == 'cutoff':
            cutoffOccurred = True
        elif result != 'failure':
            result.insert(0,childState)
            return result
    if cutoffOccurred:
        return 'cutoff'
    else:
        return 'failure'

In [8]:
def iterativeDeepeningSearch(startState, goalState, actionsF, takeActionF, maxDepth=100):
    """Calls 'depthLimitedSearch' at a range of depths up to 'maxDepth' in order to try and find the 'goalState' from
    'startState'"""
    for depth in range(maxDepth):
        result = depthLimitedSearch(startState, goalState, actionsF, takeActionF, depth)
        if result == 'failure':
            return 'failure'
        if result != 'cutoff':
            result.insert(0,startState)       
            return result
    return 'cutoff'

## 8-Tile Puzzle Functions

Most of the following functions are 8-tile puzzle functions I made from A2. There are slight modifications to both `actionsF_8p` and `takeActionF_8p`, but only to return the associated cost with the actions. They otherwise function exactly the same. A new function here is `goalTestF_8p` which determines whether or not the current state is the same as the goal state. Brief function descriptions are found within the definitions themselves.

In [9]:
def findBlank_8p(state):
    """Finds and returns the (row,column) index for the current state's blank/0 location"""
    index = state.index(0)
    row = int(index/3)
    col = index % 3
    return (row, col)

In [10]:
def actionsF_8p(state):
    """Takes the current state and returns a list of the valid actions. Four possible actions: left, right, up, down."""
    left = ('left',1)
    right= ('right',1)
    up   = ('up',1)
    down = ('down',1)
    actions = [left,right,up,down]
    blank = findBlank_8p(state)
    if(blank[0] == 0):
        if up in actions: actions.remove(up)#actions should never be removed before here, but it's good housekeeping to check
    if(blank[0] == 2):
        if down in actions: actions.remove(down)
    if(blank[1] == 0):
        if left in actions: actions.remove(left)
    if(blank[1] == 2):
        if right in actions: actions.remove(right)
    return actions

In [11]:
def takeActionF_8p(state, action):
    """Apply the given action to state and return the resulting state"""
    cp = copy.copy(state) # If a copy isn't taken, the action will permanently change the original state passed in, which is bad
    index = cp.index(0)
    left = ('left',1)
    right= ('right',1)
    up   = ('up',1)
    down = ('down',1)
    if(action == left):
        temp = cp[index-1]
        cp[index-1] = 0
        cp[index] = temp
    elif(action == right):
        temp = cp[index+1]
        cp[index+1] = 0
        cp[index] = temp
    elif(action == up):
        temp = cp[index-3]
        cp[index-3] = 0
        cp[index] = temp
    else: #action is down
        temp = cp[index+3]
        cp[index+3] = 0
        cp[index] = temp
    return (cp,1)

In [12]:
def goalTestF_8p(state, goal):
    """Returns True if state is equal to goal, false otherwise"""
    return state == goal

## Heuristic Functions

For `aStarSearch` three heuristic functions will be used. All 3 must be admissible. The first 2 were provided by the assignment and are admissible. The 3rd heuristic is one provided by me. 

  * `h1_8p(state, goal)`: $h(state, goal) = 0$, for all states $state$ and all goal states $goal$,
  * `h2_8p(state, goal)`: $h(state, goal) = m$, where $m$ is the Manhattan distance that the blank is from its goal position,
  * `h3_8p(state, goal)`: $h(state, goal) = n$, where $n$ is the number of tiles that are misplaced (excluding the blank)
  
The heuristic I'm using is in fact admissible. In the condition where the state is at the goal state, there will be no misplaced tiles. The heuristic will correctly output 0 in this case, which is less than or equal to the actual cost. Now, the blank is essentially the only 'piece' to ever move in the 8 puzzle. If a tile is misplaced, it can only be moved when the blank switches places with it. Because of these 2 facts, if 6 tiles are out of place, the blank needs to move a minimum of 6 places to try and correct those 6 tiles. My heuristic will never be able to overestimate the cost.

In [13]:
def h1_8p(state, goal):
    return 0

In [14]:
def h2_8p(state, goal):
    sRow,sCol = findBlank_8p(state)
    gRow,gCol = findBlank_8p(goal)
    m = abs(sRow-gRow) + abs(sCol-gCol)
    return m

In [15]:
def h3_8p(state, goal):
    n=0
    for i in range(len(state)):
        if state[i] != goal[i] and state[i] != 0: n+=1
    return n

## Other Functions

These last 2 functions are essentially the 'main' or 'driver' functions. The function `rumExperiment` will take in 3 goalstates as well as a list of heuristics and run `iterativeDeepeningSearch` and `aStarSearch` algorithms on all 3 goals. `aStarSearch` will be ran with each heuristic and goal for a total of (number of heuristics)\*(3 goals) times. Finally`printComparison` will be used to output a nicely formatted table summarizing the results.

In [16]:
def printComparison(goal1, goal2, goal3, results):
    """Simply formats and prints out the comparison between all 4 algorithms in this assignment"""
    print("\t{0}\t{1}\t{2}".format(goal1,goal2,goal3))
    print("{0:>9s} {1:>7s} {2:>5s} {3:>4s} {1:>21s} {2:>5s} {3:>4s} {1:>21s} {2:>5s} {3:>4s}".format("Algorithm","Nodes",
                                                                                                    "Depth","EBF"))
    print("{0:>9s} {1:^7d} {2:^5d} {3:.3f} {4:>20d} {5:^5d} {6:.3f} {7:>20d} {8:^5d} {9:.3f}".format("IDS",results[0][0],
        results[0][1],results[0][2],results[1][0],results[1][1],results[1][2],results[2][0],results[2][1],results[2][2]))   
    print("{0:>9s} {1:^7d} {2:^5d} {3:.3f} {4:>20d} {5:^5d} {6:.3f} {7:>20d} {8:^5d} {9:.3f}".format("A*h1",results[3][0],
        results[3][1],results[3][2],results[4][0],results[4][1],results[4][2],results[5][0],results[5][1],results[5][2])) 
    print("{0:>9s} {1:^7d} {2:^5d} {3:.3f} {4:>20d} {5:^5d} {6:.3f} {7:>20d} {8:^5d} {9:.3f}".format("A*h2",results[6][0],
        results[6][1],results[6][2],results[7][0],results[7][1],results[7][2],results[8][0],results[8][1],results[8][2])) 
    print("{0:>9s} {1:^7d} {2:^5d} {3:.3f} {4:>20d} {5:^5d} {6:.3f} {7:>20d} {8:^5d} {9:.3f}".format("A*h3",results[9][0],
        results[9][1],results[9][2],results[10][0],results[10][1],results[10][2],results[11][0],results[11][1],results[11][2]))

In [17]:
def runExperiment(goalState1, goalState2, goalState3, heuristics):
    """Runs iterativeDeepeningSearch and A* search with 3 provided goals. A* runs as many times as there are heuristics"""
    global nNodes
    start = [1, 2, 3, 4, 0, 5, 6, 7, 8]
    result = []
    resultInfo = []
    result = iterativeDeepeningSearch(start, goalState1, actionsF_8p, takeActionF_8p)
    depth = len(result) - 1
    resultInfo.append([nNodes, depth, ebf(nNodes, depth)])
    nNodes=0
    result = iterativeDeepeningSearch(start, goalState2, actionsF_8p, takeActionF_8p)
    depth = len(result) - 1
    resultInfo.append([nNodes, depth, ebf(nNodes, depth)])
    nNodes=0
    result = iterativeDeepeningSearch(start, goalState3, actionsF_8p, takeActionF_8p)
    depth = len(result) - 1
    resultInfo.append([nNodes, depth, ebf(nNodes, depth)])
    nNodes=0
    for heuristic in heuristics:
        result,_ = aStarSearch(start, actionsF_8p, takeActionF_8p, 
                            lambda s: goalTestF_8p(s, goalState1), lambda s: heuristic(s, goalState1))
        depth = len(result) - 1
        resultInfo.append([nNodes, depth, ebf(nNodes, depth)])
        nNodes=0
        result,_ = aStarSearch(start, actionsF_8p, takeActionF_8p, 
                            lambda s: goalTestF_8p(s, goalState2), lambda s: heuristic(s, goalState2))
        depth = len(result) - 1
        resultInfo.append([nNodes, depth, ebf(nNodes, depth)])
        nNodes=0
        result,_ = aStarSearch(start, actionsF_8p, takeActionF_8p, 
                            lambda s: goalTestF_8p(s, goalState3), lambda s: heuristic(s, goalState3))
        depth = len(result) - 1
        resultInfo.append([nNodes, depth, ebf(nNodes, depth)])
        nNodes=0
    printComparison(goalState1, goalState2, goalState3, resultInfo)

## Comparison

Four algorithms will be applied to three 8-tile puzzles with the same start. The first algorithm will be `iterativeDeepeningSearch` and the last 3 will be 3 different heuristic functions applied in `aStarSearch`.

The start state is as follows:

$$
\begin{array}{ccc}
1 & 2 & 3\\
4 & 0 & 5\\
6 & 7 & 8
\end{array}
$$

The 3 goal states are as follows:

$$
\begin{array}{ccccccccccc}
1 & 2 & 3  & ~~~~ & 1 & 2 & 3  &  ~~~~ & 1 & 0 &  3\\
4 & 0 & 5  & & 4 & 5 & 8  & & 4 & 5 & 8\\
6 & 7 & 8 &  & 6 & 0 & 7  & & 2 & 6 & 7
\end{array}
$$

In [18]:
global nNodes
nNodes=0 # Just to ensure nNodes is always set to zero before running the experiment so no issues occur when re-running this cell

g1 = [1,2,3,4,0,5,6,7,8]
g2 = [1,2,3,4,5,8,6,0,7]
g3 = [1,0,3,4,5,8,2,6,7]

results = runExperiment(g1,g2,g3,[h1_8p,h2_8p,h3_8p])

	[1, 2, 3, 4, 0, 5, 6, 7, 8]	[1, 2, 3, 4, 5, 8, 6, 0, 7]	[1, 0, 3, 4, 5, 8, 2, 6, 7]
Algorithm   Nodes Depth  EBF                 Nodes Depth  EBF                 Nodes Depth  EBF
      IDS    0      0   0.000                   43   3   3.086               225850  11   2.954
     A*h1    0      0   0.000                  116   3   4.488               643246  11   3.263
     A*h2    0      0   0.000                   51   3   3.297               100046  11   2.733
     A*h3    0      0   0.000                    9   3   1.578                 4785  11   2.031


The best heuristic appears to be the one that I came up with, A\*h3. It explores far less nodes than any of the other algorithms. In the second goal, it explores almost 5 times less nodes than the 2nd best (IDS). For goal number 3, it explores almost 21 times less nodes than the 2nd best algorithm (A\*h2). Given the growing difference between A\*h3 and the other algorithms with larger depth, it shows that a better heuristic can drastically improve runtimes.

In general though, it appears that even a relatively simple heuristic can outperform a standard iterative deepening search as the problem gets larger. For the 2nd goal, A\*h2 does slightly worse than IDS. But with goal 3, A\*h2 suddenly outperforms IDS by over a factor of 2 times. 

These results lead me to firmly believe that A\* algorithms are absolutely the way to go when it comes to path searching. This is of course assuming the heuristic paired with A\* isn't as "dumb" as A\*h1 that always returns 0. 

## Function Robustness Check

There's just 1 basic example with characters here that was provided with the assignment. It's just an additional way to ensure the search algorithms are general and can handle multiple types of data.

In [19]:
def actionsF_simple(state):
    succs = {'a': ['b', 'c'], 'b':['a'], 'c':['h'], 'h':['i'], 'i':['j', 'k', 'l'], 'k':['z']}
    return [(s, 1) for s in succs.get(state, [])]

def takeActionF_simple(state, action):
    return action

def goalTestF_simple(state, goal):
    return state == goal

def h_simple(state, goal):
    return 1

In [20]:
actions = actionsF_simple('a')
actions

[('b', 1), ('c', 1)]

In [21]:
takeActionF_simple('a', actions[0])

('b', 1)

In [22]:
goalTestF_simple('a', 'a')

True

In [23]:
h_simple('a', 'z')

1

In [24]:
iterativeDeepeningSearch('a', 'z', actionsF_simple, takeActionF_simple, 10)

['a', 'c', 'h', 'i', 'k', 'z']

In [25]:
aStarSearch('a',actionsF_simple, takeActionF_simple,
            lambda s: goalTestF_simple(s, 'z'),
            lambda s: h_simple(s, 'z'))

(['a', 'c', 'h', 'i', 'k', 'z'], 5)

## Grading

In [26]:
%run -i A3grader.py


Testing actionsF_8p([1, 2, 3, 4, 5, 6, 7, 0, 8])

--- 5/5 points. Your actionsF_8p correctly returned [('left', 1), ('right', 1), ('up', 1)]

Testing takeActionF_8p([1, 2, 3, 4, 5, 6, 7, 0, 8], (up, 1))

--- 5/5 points. Your takeActionsF_8p correctly returned ([1, 2, 3, 4, 0, 6, 7, 5, 8], 1)

Testing goalTestF_8p([1, 2, 3, 4, 5, 6, 7, 0, 8], [1, 2, 3, 4, 5, 6, 7, 0, 8])

--- 5/5 points. Your goalTestF_8p correctly True

Testing aStarSearch([1, 2, 3, 4, 5, 6, 7, 0, 8],
                     actionsF_8p, takeActionF_8p,
                     lambda s: goalTestF_8p(s, [0, 2, 3, 1, 4,  6, 7, 5, 8]),
                     lambda s: h1_8p(s, [0, 2, 3, 1, 4,  6, 7, 5, 8]))

--- 20/20 points. Your search correctly returned ([[1, 2, 3, 4, 5, 6, 7, 0, 8], [1, 2, 3, 4, 0, 6, 7, 5, 8], [1, 2, 3, 0, 4, 6, 7, 5, 8], [0, 2, 3, 1, 4, 6, 7, 5, 8]], 3)

Testing iterativeDeepeningSearch([5, 2, 8, 0, 1, 4, 3, 7, 6], 
                                 [0, 2, 3, 1, 4,  6, 7, 5, 8],
                            